In [25]:
from flash_ansr.expressions import ExpressionSpace
from flash_ansr.expressions.utils import codify, num_to_constants
from flash_ansr import get_path
import itertools
from tqdm import tqdm
from collections import defaultdict
import numpy as np
from typing import Generator, Callable
import warnings

In [26]:
MODEL = 'v7.0'

In [27]:
config = get_path('configs', MODEL, 'expression_space.yaml')

In [28]:
space = ExpressionSpace.from_config(config)

In [29]:
leaf_nodes = space.variables + ["<num>"]
non_leaf_nodes = space.operator_arity
non_leaf_nodes = dict(sorted(non_leaf_nodes.items(), key=lambda x: x[1]))

print(leaf_nodes)
print(non_leaf_nodes)

['x1', 'x2', 'x3', '<num>']
{'neg': 1, 'abs': 1, 'inv': 1, 'pow2': 1, 'pow3': 1, 'pow4': 1, 'pow5': 1, 'pow1_2': 1, 'pow1_3': 1, 'pow1_4': 1, 'pow1_5': 1, 'sin': 1, 'cos': 1, 'tan': 1, 'asin': 1, 'acos': 1, 'atan': 1, 'exp': 1, 'log': 1, '+': 2, '-': 2, '*': 2, '/': 2}


In [30]:
def apply_rule(X, A, B):
    result = []
    i = 0
    while i < len(X):
        # Check if sublist A is found at current position
        if i <= len(X) - len(A) and X[i:i+len(A)] == A:
            # Add replacement sublist B
            result.extend(B)
            # Skip past the matched sublist A
            i += len(A)
        else:
            # Add current element and move to next
            result.append(X[i])
            i += 1
    return result

# Example usage
X = ['+', 'cos', 'x', 'sin', '*', '*', 'x', 'x', '*', 'x', 'x']
A = ['*', 'x', 'x']
B = ['pow2', 'x']
print(apply_rule(X, A, B))


['+', 'cos', 'x', 'sin', '*', 'pow2', 'x', 'pow2', 'x']


In [31]:
def simplify(expression: list[str], rules: set[tuple[list[str], list[str]]]):
    if isinstance(expression, tuple):
        expression = list(expression)
    for pattern, replacement in rules:
        expression = apply_rule(expression, pattern, replacement)
    return expression

In [32]:
simplify(['+', 'cos', 'x', 'sin', '*', '*', 'x', 'x', '*', 'x', 'x'], [(['*', 'x', 'x'], ['pow2', 'x'])])

['+', 'cos', 'x', 'sin', '*', 'pow2', 'x', 'pow2', 'x']

In [33]:
simplify(['neg', '-', 'x1', 'x1'], [(['-', 'x1', 'x1'], ['0'])])

['neg', '0']

In [34]:
rules = []

In [35]:
simplify(['<num>'], rules)

['<num>']

In [36]:
def expression_generator(hashes_of_size: dict[int, list[tuple[str]]], non_leaf_nodes: dict[str, int]) -> Generator[tuple[str], None, None]:
    # Append existing trees to every operator
    for new_root_operator, arity in non_leaf_nodes.items():
        # Start with the smallest arity-tuples of trees
        for child_lengths in sorted(itertools.product(list(hashes_of_size.keys()), repeat=arity), key=lambda x: sum(x)):
            # Check all possible combinations of child trees
            for child_combination in itertools.product(*[hashes_of_size[child_length] for child_length in child_lengths]):
                yield (new_root_operator,) + tuple(itertools.chain.from_iterable(child_combination))

In [37]:
X = np.random.normal(loc=0, scale=5, size=(1024, space.n_variables))

In [38]:
size = 0
rules = []

hashes_of_size = defaultdict(list)

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=RuntimeWarning)

    # Create all leaf nodes
    for leaf in leaf_nodes[:size]:
        simplified_skeleton = simplify([leaf], rules)
        
        executable_prefix_expression = space.operators_to_realizations(simplified_skeleton)
        prefix_expression_with_constants, constants = num_to_constants(executable_prefix_expression)
        code_string = space.prefix_to_infix(prefix_expression_with_constants, realization=True)
        code = codify(code_string, space.variables + constants)

        hashes_of_size[len(simplified_skeleton)].append(tuple(simplified_skeleton))

    pbar = tqdm(total=size)
    n_scanned = 0

    while n_scanned < size:
        simplified_hashes_of_size = defaultdict(list)
        for l, hashes_list in hashes_of_size.items():
            for h in hashes_list:
                simplified_skeleton = simplify(h, rules)
                simplified_hashes_of_size[len(simplified_skeleton)].append(simplified_skeleton)
        hashes_of_size = simplified_hashes_of_size

        new_hashes_of_size = defaultdict(list)
        for combination in expression_generator(hashes_of_size, non_leaf_nodes):
            for i, rule in enumerate(rules):
                rules[i] = (rule[0], simplify(rule[1], rules))

            simplified_skeleton = simplify(list(combination), rules)
            h = tuple(simplified_skeleton)

            executable_prefix_expression = space.operators_to_realizations(simplified_skeleton)
            prefix_expression_with_constants, constants = num_to_constants(executable_prefix_expression)
            code_string = space.prefix_to_infix(prefix_expression_with_constants, realization=True)
            code = codify(code_string, space.variables + constants)

            # Record the image
            if len(constants) == 0:
                f = space.code_to_lambda(code)
                y = f(*X.T)

                for candidate_hashes_of_size in (hashes_of_size, new_hashes_of_size):
                    for l, candidate_hashes_list in candidate_hashes_of_size.items():
                        # Ignore simplification candidates that do not shorten the expression
                        if l >= len(h):
                            continue

                        for candidate_hash in candidate_hashes_list:
                            if candidate_hash == h:
                                continue
                            executable_prefix_candidate_hash = space.operators_to_realizations(candidate_hash)
                            prefix_candidate_hash_with_constants, constants_candidate_hash = num_to_constants(executable_prefix_candidate_hash)
                            code_string_candidate_hash = space.prefix_to_infix(prefix_candidate_hash_with_constants, realization=True)
                            code_candidate_hash = codify(code_string_candidate_hash, space.variables + constants_candidate_hash)

                            # Record the image
                            if len(constants_candidate_hash) == 0:
                                f_candidate = space.code_to_lambda(code_candidate_hash)
                                y_candidate = f_candidate(*X.T)

                                if np.allclose(y, y_candidate, equal_nan=True):
                                    rules.append((simplified_skeleton, list(candidate_hash)))

            new_hashes_of_size[len(h)].append(h)

            n_scanned += 1
            pbar.update(1)
            pbar.set_postfix_str(f"Rules found: {len(rules):,}")

            if n_scanned >= size:
                break

        hashes_of_size.update(new_hashes_of_size)

    pbar.close()

# Write the rules to a file
with open('./rules.txt', 'w') as f:
    for rule in rules:
        f.write(f"{rule[0]} -> {rule[1]}\n")

  0%|          | 0/1000 [00:31<?, ?it/s, Rules found: 0, Current Expression: ('neg', 'x1')]
0it [00:00, ?it/s]


In [39]:
from scipy.optimize import curve_fit, OptimizeWarning

In [40]:
def exist_constants_that_fit(expression: list[str], X: np.ndarray, y_target: np.ndarray):
    if isinstance(expression, tuple):
        expression = list(expression)

    executable_prefix_expression = space.operators_to_realizations(expression)
    prefix_expression_with_constants, constants = num_to_constants(executable_prefix_expression, convert_numbers_to_constant=False)
    code_string = space.prefix_to_infix(prefix_expression_with_constants, realization=True)
    code = codify(code_string, space.variables + constants)
    f = space.code_to_lambda(code)

    def pred_function(X: np.ndarray, *constants: np.ndarray | None) -> float:
        if len(constants) == 0:
            return f(*X.T)
        return f(*X.T, *constants)

    p0 = np.random.normal(loc=0, scale=5, size=len(constants))

    is_valid = np.isfinite(X).all(axis=1) & np.isfinite(y_target)

    if not np.any(is_valid):
        return False

    try:
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=OptimizeWarning)
            popt, _ = curve_fit(pred_function, X[is_valid], y_target[is_valid].flatten(), p0=p0)
    except RuntimeError:
        return False

    y = f(*X.T, *popt)
    if not isinstance(y, np.ndarray):
        y = np.full(X.shape[0], y)

    return np.allclose(y_target, y, equal_nan=True)

In [41]:
C = np.random.normal(loc=0, scale=5, size=(1024, 128))
X_with_constants = np.hstack((X, C))

In [42]:
def safe_f(f, X):
    try:
        return f(*X.T)
    except ZeroDivisionError:
        return np.full(X.shape[0], np.nan)

In [21]:
size = 10_000
constants_retries = 5
rules = [(['-', t, t], ['0']) for t in space.variables] + [(['/', t, t], ['1']) for t in space.variables] + [(['*', t, '1'], [t]) for t in space.variables]

hashes_of_size = defaultdict(list)

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=RuntimeWarning)

    # Create all leaf nodes
    for leaf in leaf_nodes[:size]:
        simplified_skeleton = simplify([leaf], rules)
        
        executable_prefix_expression = space.operators_to_realizations(simplified_skeleton)
        prefix_expression_with_constants, constants = num_to_constants(executable_prefix_expression)
        code_string = space.prefix_to_infix(prefix_expression_with_constants, realization=True)
        code = codify(code_string, space.variables + constants)

        hashes_of_size[len(simplified_skeleton)].append(tuple(simplified_skeleton))

    pbar = tqdm(total=size)
    n_scanned = 0

    while n_scanned < size:
        simplified_hashes_of_size = defaultdict(list)
        for l, hashes_list in hashes_of_size.items():
            for h in hashes_list:
                simplified_skeleton = simplify(h, rules)
                simplified_hashes_of_size[len(simplified_skeleton)].append(simplified_skeleton)
        hashes_of_size = simplified_hashes_of_size

        new_hashes_of_size = defaultdict(list)
        for combination in expression_generator(hashes_of_size, non_leaf_nodes):
            pbar.set_postfix_str(f"Rules found: {len(rules):,}, Current Expression: {combination}")
            for i, rule in enumerate(rules):
                rules[i] = (rule[0], simplify(rule[1], rules))

            simplified_skeleton = simplify(list(combination), rules)
            h = tuple(simplified_skeleton)

            executable_prefix_expression = space.operators_to_realizations(simplified_skeleton)
            prefix_expression_with_constants, constants = num_to_constants(executable_prefix_expression)
            code_string = space.prefix_to_infix(prefix_expression_with_constants, realization=True)
            code = codify(code_string, space.variables + constants)
            
            f = space.code_to_lambda(code)

            # Record the image
            if len(constants) == 0:
                y = f(*X.T)

                for candidate_hashes_of_size in (hashes_of_size, new_hashes_of_size):
                    for l, candidate_hashes_list in candidate_hashes_of_size.items():
                        # Ignore simplification candidates that do not shorten the expression
                        if l >= len(h):
                            continue

                        for candidate_hash in candidate_hashes_list:
                            if candidate_hash == h:
                                continue
                            executable_prefix_candidate_hash = space.operators_to_realizations(candidate_hash)
                            prefix_candidate_hash_with_constants, constants_candidate_hash = num_to_constants(executable_prefix_candidate_hash)
                            code_string_candidate_hash = space.prefix_to_infix(prefix_candidate_hash_with_constants, realization=True)
                            code_candidate_hash = codify(code_string_candidate_hash, space.variables + constants_candidate_hash)

                            # Record the image
                            if len(constants_candidate_hash) == 0:
                                f_candidate = space.code_to_lambda(code_candidate_hash)
                                y_candidate = f_candidate(*X.T)

                                if np.allclose(y, y_candidate, equal_nan=True):
                                    rules.append((simplified_skeleton, list(candidate_hash)))
                            else:
                                if any([exist_constants_that_fit(candidate_hash, X, y) for _ in range(constants_retries)]):
                                    rules.append((simplified_skeleton, list(candidate_hash)))
            else:
                # Create an image from X and randomly sampled constants
                y = f(*X_with_constants[:, :len(space.variables) + len(constants)].T)

                for candidate_hashes_of_size in (hashes_of_size, new_hashes_of_size):
                    for l, candidate_hashes_list in candidate_hashes_of_size.items():
                        # Ignore simplification candidates that do not shorten the expression
                        if l >= len(h):
                            continue

                        for candidate_hash in candidate_hashes_list:
                            if candidate_hash == h:
                                continue
                            executable_prefix_candidate_hash = space.operators_to_realizations(candidate_hash)
                            prefix_candidate_hash_with_constants, constants_candidate_hash = num_to_constants(executable_prefix_candidate_hash)
                            code_string_candidate_hash = space.prefix_to_infix(prefix_candidate_hash_with_constants, realization=True)
                            code_candidate_hash = codify(code_string_candidate_hash, space.variables + constants_candidate_hash)

                            f_candidate = space.code_to_lambda(code_candidate_hash)

                            # Record the image
                            if len(constants_candidate_hash) == 0:
                                y_candidate = f_candidate(*X.T)

                                if np.allclose(y, y_candidate, equal_nan=True):
                                    rules.append((simplified_skeleton, list(candidate_hash)))
                            else:
                                if any([exist_constants_that_fit(candidate_hash, X, y) for _ in range(constants_retries)]):
                                    rules.append((simplified_skeleton, list(candidate_hash)))

            new_hashes_of_size[len(h)].append(h)

            n_scanned += 1
            pbar.update(1)

            if n_scanned >= size:
                break

        hashes_of_size.update(new_hashes_of_size)

    pbar.close()

# Write the rules to a file
with open('./rules_constants.txt', 'w') as f:
    for rule in rules:
        f.write(f"{rule[0]} -> {rule[1]}\n")

 15%|█▌        | 154/1000 [00:01<00:11, 70.94it/s, Rules found: 47, Current Expression: ('-', 'x1', '0') -> ['-', 'x1', '0'] -> ...]               

('-', 'x1', 'x1')
['<num>'] ['C_0'] ['C_0'] C_0
['0'] ['0'] [] 0
['nsrops.neg', '<num>'] ['nsrops.neg', 'C_0'] ['C_0'] nsrops.neg(C_0)
['nsrops.neg', '0'] ['nsrops.neg', '0'] [] nsrops.neg(0)
['abs', '<num>'] ['abs', 'C_0'] ['C_0'] abs(C_0)
['abs', '0'] ['abs', '0'] [] abs(0)
['nsrops.pow2', '0'] ['nsrops.pow2', '0'] [] nsrops.pow2(0)
['nsrops.pow3', '0'] ['nsrops.pow3', '0'] [] nsrops.pow3(0)
['nsrops.pow4', '0'] ['nsrops.pow4', '0'] [] nsrops.pow4(0)
['nsrops.pow5', '0'] ['nsrops.pow5', '0'] [] nsrops.pow5(0)
['nsrops.pow1_2', '0'] ['nsrops.pow1_2', '0'] [] nsrops.pow1_2(0)
['nsrops.pow1_3', '0'] ['nsrops.pow1_3', '0'] [] nsrops.pow1_3(0)
['nsrops.pow1_4', '0'] ['nsrops.pow1_4', '0'] [] nsrops.pow1_4(0)
['nsrops.pow1_5', '0'] ['nsrops.pow1_5', '0'] [] nsrops.pow1_5(0)
['numpy.sin', '<num>'] ['numpy.sin', 'C_0'] ['C_0'] numpy.sin(C_0)
['numpy.sin', '0'] ['numpy.sin', '0'] [] numpy.sin(0)
['numpy.cos', '<num>'] ['numpy.cos', 'C_0'] ['C_0'] numpy.cos(C_0)
['numpy.tan', '<num>'] ['numpy.

100%|██████████| 1000/1000 [02:19<00:00,  7.19it/s, Rules found: 169, Current Expression: ('inv', '-', 'x3', 'x1') -> ['inv', '-', 'x3', 'x1'] -> ...]           


In [24]:
size = 1_000
constants_retries = 5
rules = []

hashes_of_size = defaultdict(list)

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=RuntimeWarning)

    # Create all leaf nodes
    for leaf in leaf_nodes[:size]:
        simplified_skeleton = simplify([leaf], rules)
        
        executable_prefix_expression = space.operators_to_realizations(simplified_skeleton)
        prefix_expression_with_constants, constants = num_to_constants(executable_prefix_expression)
        code_string = space.prefix_to_infix(prefix_expression_with_constants, realization=True)
        code = codify(code_string, space.variables + constants)

        hashes_of_size[len(simplified_skeleton)].append(tuple(simplified_skeleton))

    pbar = tqdm(total=size)
    n_scanned = 0

    while n_scanned < size:
        simplified_hashes_of_size = defaultdict(list)
        for l, hashes_list in hashes_of_size.items():
            for h in hashes_list:
                simplified_skeleton = simplify(h, rules)
                simplified_hashes_of_size[len(simplified_skeleton)].append(simplified_skeleton)
        hashes_of_size = simplified_hashes_of_size

        new_hashes_of_size = defaultdict(list)
        for combination in expression_generator(hashes_of_size, non_leaf_nodes):
            for i, rule in enumerate(rules):
                rules[i] = (rule[0], simplify(rule[1], rules))

            simplified_skeleton = simplify(list(combination), rules)
            h = tuple(simplified_skeleton)

            pbar.set_postfix_str(f"Rules found: {len(rules):,}, Current Expression: {combination} -> {simplified_skeleton} -> ...")

            executable_prefix_expression = space.operators_to_realizations(simplified_skeleton)
            prefix_expression_with_constants, constants = num_to_constants(executable_prefix_expression)
            code_string = space.prefix_to_infix(prefix_expression_with_constants, realization=True)
            code = codify(code_string, space.variables + constants)
            
            f = space.code_to_lambda(code)

            # Record the image
            if len(constants) == 0:
                y = f(*X.T)

                for candidate_hashes_of_size in (hashes_of_size, new_hashes_of_size):
                    for l, candidate_hashes_list in candidate_hashes_of_size.items():
                        # Ignore simplification candidates that do not shorten the expression
                        if l >= len(h):
                            continue

                        for candidate_hash in candidate_hashes_list:
                            if candidate_hash == h:
                                continue
                            executable_prefix_candidate_hash = space.operators_to_realizations(candidate_hash)
                            prefix_candidate_hash_with_constants, constants_candidate_hash = num_to_constants(executable_prefix_candidate_hash)
                            code_string_candidate_hash = space.prefix_to_infix(prefix_candidate_hash_with_constants, realization=True)
                            code_candidate_hash = codify(code_string_candidate_hash, space.variables + constants_candidate_hash)

                            # Record the image
                            if len(constants_candidate_hash) == 0:
                                f_candidate = space.code_to_lambda(code_candidate_hash)
                                y_candidate = f_candidate(*X.T)

                                if np.allclose(y, y_candidate, equal_nan=True):
                                    rules.append((simplified_skeleton, list(candidate_hash)))
                            else:
                                if any([exist_constants_that_fit(candidate_hash, X, y) for _ in range(constants_retries)]):
                                    rules.append((simplified_skeleton, list(candidate_hash)))
            else:
                # Create an image from X and randomly sampled constants
                y = f(*X_with_constants[:, :len(space.variables) + len(constants)].T)

                for candidate_hashes_of_size in (hashes_of_size, new_hashes_of_size):
                    for l, candidate_hashes_list in candidate_hashes_of_size.items():
                        # Ignore simplification candidates that do not shorten the expression
                        if l >= len(h):
                            continue

                        for candidate_hash in candidate_hashes_list:
                            if candidate_hash == h:
                                continue
                            executable_prefix_candidate_hash = space.operators_to_realizations(candidate_hash)
                            prefix_candidate_hash_with_constants, constants_candidate_hash = num_to_constants(executable_prefix_candidate_hash)
                            code_string_candidate_hash = space.prefix_to_infix(prefix_candidate_hash_with_constants, realization=True)
                            code_candidate_hash = codify(code_string_candidate_hash, space.variables + constants_candidate_hash)

                            f_candidate = space.code_to_lambda(code_candidate_hash)

                            # Record the image
                            if len(constants_candidate_hash) == 0:
                                y_candidate = f_candidate(*X.T)

                                if np.allclose(y, y_candidate, equal_nan=True):
                                    rules.append((simplified_skeleton, list(candidate_hash)))
                            else:
                                if any([exist_constants_that_fit(candidate_hash, X, y) for _ in range(constants_retries)]):
                                    rules.append((simplified_skeleton, list(candidate_hash)))

            new_hashes_of_size[len(h)].append(h)

            n_scanned += 1
            pbar.update(1)

            if n_scanned >= size:
                break

        hashes_of_size.update(new_hashes_of_size)

    pbar.close()

# Write the rules to a file
with open('./rules_constants.txt', 'w') as f:
    for rule in rules:
        f.write(f"{rule[0]} -> {rule[1]}\n")

 53%|█████▎    | 528/1000 [00:28<01:29,  5.29it/s, Rules found: 148, Current Expression: ('inv', '+', 'x3', '<num>')]   

In [ ]:
size = 1_000
constants_retries = 5
rules = []

hashes_of_size = defaultdict(list)

leaf_nodes = space.variables + ["<num>"] + ['0', '1']

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=RuntimeWarning)

    # Create all leaf nodes
    for leaf in leaf_nodes[:size]:
        simplified_skeleton = simplify([leaf], rules)
        
        executable_prefix_expression = space.operators_to_realizations(simplified_skeleton)
        prefix_expression_with_constants, constants = num_to_constants(executable_prefix_expression, convert_numbers_to_constant=False)
        code_string = space.prefix_to_infix(prefix_expression_with_constants, realization=True)
        code = codify(code_string, space.variables + constants)

        hashes_of_size[len(simplified_skeleton)].append(tuple(simplified_skeleton))

    pbar = tqdm(total=size)
    n_scanned = 0

    while n_scanned < size:
        simplified_hashes_of_size = defaultdict(list)
        for l, hashes_list in hashes_of_size.items():
            for h in hashes_list:
                simplified_skeleton = simplify(h, rules)
                simplified_hashes_of_size[len(simplified_skeleton)].append(simplified_skeleton)
        hashes_of_size = simplified_hashes_of_size

        new_hashes_of_size = defaultdict(list)
        for combination in expression_generator(hashes_of_size, non_leaf_nodes):
            if combination == ('-', 'x1', 'x1'):
                print(combination)
            for i, rule in enumerate(rules):
                rules[i] = (rule[0], simplify(rule[1], rules))

            simplified_skeleton = simplify(list(combination), rules)
            h = tuple(simplified_skeleton)

            pbar.set_postfix_str(f"Rules found: {len(rules):,}, Current Expression: {combination} -> {simplified_skeleton} -> ...")

            executable_prefix_expression = space.operators_to_realizations(simplified_skeleton)
            prefix_expression_with_constants, constants = num_to_constants(executable_prefix_expression, convert_numbers_to_constant=False)
            code_string = space.prefix_to_infix(prefix_expression_with_constants, realization=True)
            code = codify(code_string, space.variables + constants)
            
            f = space.code_to_lambda(code)

            # Record the image
            if len(constants) == 0:
                y = safe_f(f, X)
                if not isinstance(y, np.ndarray):
                    y = np.full(X.shape[0], y)

                new_rule_candidates = []
                for candidate_hashes_of_size in (hashes_of_size, new_hashes_of_size):
                    for l, candidate_hashes_list in candidate_hashes_of_size.items():
                        # Ignore simplification candidates that do not shorten the expression
                        if l >= len(h):
                            continue

                        for candidate_hash in candidate_hashes_list:
                            if candidate_hash == h:
                                continue
                            executable_prefix_candidate_hash = space.operators_to_realizations(candidate_hash)
                            prefix_candidate_hash_with_constants, constants_candidate_hash = num_to_constants(executable_prefix_candidate_hash, convert_numbers_to_constant=False)
                            code_string_candidate_hash = space.prefix_to_infix(prefix_candidate_hash_with_constants, realization=True)
                            code_candidate_hash = codify(code_string_candidate_hash, space.variables + constants_candidate_hash)

                            # print(prefix_candidate_hash_with_constants, constants_candidate_hash, code_string_candidate_hash)
                            

                            # Record the image
                            if len(constants_candidate_hash) == 0:
                                f_candidate = space.code_to_lambda(code_candidate_hash)
                                y_candidate = safe_f(f_candidate, X)
                                if not isinstance(y_candidate, np.ndarray):
                                    y_candidate = np.full(X.shape[0], y_candidate)

                                if np.allclose(y, y_candidate, equal_nan=True):
                                    
                                    if combination == ('-', 'x1', 'x1'):
                                        print(executable_prefix_candidate_hash, prefix_candidate_hash_with_constants, constants_candidate_hash, code_string_candidate_hash)
                                    new_rule_candidates.append((simplified_skeleton, list(candidate_hash)))
                            else:
                                if any([exist_constants_that_fit(candidate_hash, X, y) for _ in range(constants_retries)]):
                                    if combination == ('-', 'x1', 'x1'):
                                        print(executable_prefix_candidate_hash, prefix_candidate_hash_with_constants, constants_candidate_hash, code_string_candidate_hash)
                                    new_rule_candidates.append((simplified_skeleton, list(candidate_hash)))
                        
                # Find the shortest rule
                if len(new_rule_candidates) > 0:
                    new_rule_candidates = sorted(new_rule_candidates, key=lambda x: len(x[1]))
                    rules.append(new_rule_candidates[0])

            else:
                # Create an image from X and randomly sampled constants
                y = f(*X_with_constants[:, :len(space.variables) + len(constants)].T)

                new_rule_candidates = []
                for candidate_hashes_of_size in (hashes_of_size, new_hashes_of_size):
                    for l, candidate_hashes_list in candidate_hashes_of_size.items():
                        # Ignore simplification candidates that do not shorten the expression
                        if l >= len(h):
                            continue

                        for candidate_hash in candidate_hashes_list:
                            if candidate_hash == h:
                                continue
                            executable_prefix_candidate_hash = space.operators_to_realizations(candidate_hash)
                            prefix_candidate_hash_with_constants, constants_candidate_hash = num_to_constants(executable_prefix_candidate_hash, convert_numbers_to_constant=False)
                            code_string_candidate_hash = space.prefix_to_infix(prefix_candidate_hash_with_constants, realization=True)
                            code_candidate_hash = codify(code_string_candidate_hash, space.variables + constants_candidate_hash)

                            f_candidate = space.code_to_lambda(code_candidate_hash)
                            
                            # Record the image
                            if len(constants_candidate_hash) == 0:
                                y_candidate = safe_f(f_candidate, X)
                                if not isinstance(y_candidate, np.ndarray):
                                    y_candidate = np.full(X.shape[0], y_candidate)

                                if np.allclose(y, y_candidate, equal_nan=True):
                                    new_rule_candidates.append((simplified_skeleton, list(candidate_hash)))
                            else:
                                if any([exist_constants_that_fit(candidate_hash, X, y) for _ in range(constants_retries)]):
                                    new_rule_candidates.append((simplified_skeleton, list(candidate_hash)))

                # Find the shortest rule
                if len(new_rule_candidates) > 0:
                    new_rule_candidates = sorted(new_rule_candidates, key=lambda x: len(x[1]))
                    rules.append(new_rule_candidates[0])

            new_hashes_of_size[len(h)].append(h)

            n_scanned += 1
            pbar.update(1)

            if n_scanned >= size:
                break

        hashes_of_size.update(new_hashes_of_size)

    pbar.close()

# Write the rules to a file
with open('./rules_constants.txt', 'w') as f:
    for rule in rules:
        f.write(f"{rule[0]} -> {rule[1]}\n")

 15%|█▌        | 154/1000 [00:01<00:11, 70.94it/s, Rules found: 47, Current Expression: ('-', 'x1', '0') -> ['-', 'x1', '0'] -> ...]               

('-', 'x1', 'x1')
['<num>'] ['C_0'] ['C_0'] C_0
['0'] ['0'] [] 0
['nsrops.neg', '<num>'] ['nsrops.neg', 'C_0'] ['C_0'] nsrops.neg(C_0)
['nsrops.neg', '0'] ['nsrops.neg', '0'] [] nsrops.neg(0)
['abs', '<num>'] ['abs', 'C_0'] ['C_0'] abs(C_0)
['abs', '0'] ['abs', '0'] [] abs(0)
['nsrops.pow2', '0'] ['nsrops.pow2', '0'] [] nsrops.pow2(0)
['nsrops.pow3', '0'] ['nsrops.pow3', '0'] [] nsrops.pow3(0)
['nsrops.pow4', '0'] ['nsrops.pow4', '0'] [] nsrops.pow4(0)
['nsrops.pow5', '0'] ['nsrops.pow5', '0'] [] nsrops.pow5(0)
['nsrops.pow1_2', '0'] ['nsrops.pow1_2', '0'] [] nsrops.pow1_2(0)
['nsrops.pow1_3', '0'] ['nsrops.pow1_3', '0'] [] nsrops.pow1_3(0)
['nsrops.pow1_4', '0'] ['nsrops.pow1_4', '0'] [] nsrops.pow1_4(0)
['nsrops.pow1_5', '0'] ['nsrops.pow1_5', '0'] [] nsrops.pow1_5(0)
['numpy.sin', '<num>'] ['numpy.sin', 'C_0'] ['C_0'] numpy.sin(C_0)
['numpy.sin', '0'] ['numpy.sin', '0'] [] numpy.sin(0)
['numpy.cos', '<num>'] ['numpy.cos', 'C_0'] ['C_0'] numpy.cos(C_0)
['numpy.tan', '<num>'] ['numpy.

100%|██████████| 1000/1000 [02:19<00:00,  7.19it/s, Rules found: 169, Current Expression: ('inv', '-', 'x3', 'x1') -> ['inv', '-', 'x3', 'x1'] -> ...]           


In [19]:
pbar.close()

 25%|██▌       | 250/1000 [00:23<01:09, 10.84it/s, Rules found: 90, Current Expression: ('/', '0', '0') -> ['/', '0', '0'] -> ...]


In [21]:
import json

In [22]:
with open('./rules_constants.json', 'w') as f:
    json.dump(rules, f, indent=4)